In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:1"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 4, ndecoderlayers = 2, dropout = 0):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1):
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints, 2], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints, 2)
        )
        
        self.distance_matrix = (self.points.unsqueeze(1) - self.points.unsqueeze(2)).square().sum(-1).sqrt() # [batchsize * nsamples, npoints, npoints]
        
        self.previous_point = None
        
        self.points_mask = torch.zeros(
                    [self.batchsize, npoints], dtype=torch.bool, device=device
                )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert list(point_index.size()) == [self.batchsize]
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        if self.previous_point != None:
            self.cost += self.distance_matrix.gather(2, self.previous_point.unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, point_index.unsqueeze(1)).squeeze(1)
        
        self.previous_point = point_index
        self.points_mask.scatter_(1, point_index.unsqueeze(1), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
        
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    
    def laststep(self):
        
        assert self.points_sequence.size(1) == self.npoints
        
        self.cost += self.distance_matrix.gather(2, self.points_sequence[:, 0].unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, self.points_sequence[:, -1].unsqueeze(1)).squeeze(1)
    

env = environment()


def evaluate(epochs = 10, npoints = 10, batchsize = 100, nsamples = 2, negative_cutoff = 1):
    NN.eval()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item())


def train(epochs = 30000, npoints = 10, batchsize = 100, nsamples = 8, negative_cutoff = 1):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        
        NN.eval()
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 0.5
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        #loss = ( 
        #        - logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
        #        * positive_reinforcement_binary 
        #        + logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 40) 
        #        * negative_reinforcement_binary 
        #).mean()
        """
        scaled binary reinforcement
        """
        #loss = ( 
        #        (- logprob.view(batchsize, nsamples)[:, :-1] 
        #        * positive_reinforcement_binary).sum() / (positive_reinforcement_binary.sum() + 1)
        #        + (logprob.view(batchsize, nsamples)[:, :-1] 
        #        * negative_reinforcement_binary).sum() / (negative_reinforcement_binary.sum() + 1)
        #)
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
   





In [2]:
train(300000, 100, 150, 8, 1)

43.59682083129883 -360.5675354003906 -363.66595458984375 -363.6621398925781 -363.7589416503906 -363.7590637207031
25.585384368896484 -350.68408203125 -362.6767883300781 -365.5819091796875 -363.553466796875 -363.553466796875
29.714935302734375 -328.89910888671875 -355.6352844238281 -360.91656494140625 -353.8456726074219 -353.845703125
30.67466163635254 -277.6025085449219 -320.9776916503906 -316.4792175292969 -323.41583251953125 -323.41583251953125
29.225189208984375 -239.4132843017578 -283.32568359375 -278.99676513671875 -289.76202392578125 -289.7621154785156
27.416074752807617 -235.74549865722656 -278.38970947265625 -278.9473876953125 -267.3858947753906 -267.3858642578125
29.190990447998047 -229.38449096679688 -273.9106750488281 -263.54656982421875 -274.17938232421875 -274.1793518066406
29.031681060791016 -234.57130432128906 -275.7728576660156 -269.7939453125 -268.46038818359375 -268.4603271484375
28.97440528869629 -231.1998748779297 -273.657958984375 -281.79779052734375 -258.229064941

11.187114715576172 -61.76106262207031 -103.11656188964844 -98.43328094482422 -104.21669006347656 -104.21668243408203
11.29931354522705 -62.06776809692383 -104.34654998779297 -107.17308044433594 -97.82612609863281 -97.82608795166016
11.213188171386719 -61.782047271728516 -104.63780975341797 -106.18241882324219 -108.36970520019531 -108.36970520019531
10.98259162902832 -60.36834716796875 -101.68559265136719 -107.88016510009766 -114.19603729248047 -114.196044921875
10.993087768554688 -60.083740234375 -100.22916412353516 -83.15007019042969 -112.96762084960938 -112.96765899658203
11.089509963989258 -61.3326530456543 -100.9887924194336 -101.21658325195312 -92.16109466552734 -92.16111755371094
11.128193855285645 -62.671817779541016 -101.89512634277344 -117.72380828857422 -107.76435089111328 -107.764404296875
11.050711631774902 -61.29323196411133 -101.0897216796875 -99.69673156738281 -90.21751403808594 -90.21749114990234
11.252094268798828 -61.76125717163086 -101.59477996826172 -96.312316894531

9.974191665649414 -34.03178405761719 -62.051025390625 -69.26712799072266 -73.52935028076172 -73.52932739257812
9.842011451721191 -33.42542266845703 -61.874298095703125 -62.85533142089844 -66.27226257324219 -66.27223205566406
9.950874328613281 -33.87977981567383 -62.405731201171875 -57.1097412109375 -63.059146881103516 -63.05915069580078
9.901421546936035 -34.20343017578125 -63.39704513549805 -61.949214935302734 -63.852508544921875 -63.8525390625
9.735060691833496 -33.10398483276367 -61.54052734375 -52.90763473510742 -62.05125045776367 -62.05124282836914
9.643346786499023 -32.568355560302734 -60.72285079956055 -69.80633544921875 -58.114051818847656 -58.114044189453125
9.675130844116211 -31.84825897216797 -59.94277572631836 -65.00485229492188 -61.67154312133789 -61.671531677246094
9.611139297485352 -32.3681640625 -60.128089904785156 -62.70527267456055 -62.82469940185547 -62.824710845947266
9.706106185913086 -32.104488372802734 -59.621437072753906 -57.09444808959961 -59.54237747192383 -59

9.640585899353027 -25.756086349487305 -47.673160552978516 -48.49714279174805 -50.96650695800781 -50.96650695800781
9.719207763671875 -25.845386505126953 -48.151065826416016 -46.363609313964844 -42.830501556396484 -42.83052444458008
9.702227592468262 -25.3500919342041 -47.90387725830078 -40.36891555786133 -41.35061264038086 -41.35066604614258
9.707112312316895 -25.547834396362305 -48.36753463745117 -47.209869384765625 -41.53559494018555 -41.53559875488281
9.600193977355957 -25.384483337402344 -48.659969329833984 -53.67289733886719 -43.92365646362305 -43.92361068725586
9.581022262573242 -25.469560623168945 -48.68178939819336 -45.269954681396484 -55.30572509765625 -55.30570983886719
9.628417015075684 -26.332529067993164 -49.69792175292969 -43.49345397949219 -53.96370315551758 -53.963661193847656
9.687560081481934 -26.125164031982422 -49.855220794677734 -55.202457427978516 -43.90903091430664 -43.90900421142578
9.679656982421875 -26.273902893066406 -50.20343780517578 -38.15265655517578 -42.

9.371084213256836 -21.38071060180664 -41.16659164428711 -35.502044677734375 -44.76598358154297 -44.76599884033203
9.38515853881836 -21.072187423706055 -40.95290756225586 -46.59735107421875 -38.376583099365234 -38.37659454345703
9.457530975341797 -20.919239044189453 -40.425846099853516 -58.3310661315918 -41.42876434326172 -41.42880630493164
9.553190231323242 -21.042125701904297 -40.47024154663086 -45.31163024902344 -36.2777099609375 -36.277687072753906
9.548772811889648 -20.88405418395996 -39.60627746582031 -26.593036651611328 -30.849700927734375 -30.849647521972656
9.471858024597168 -21.21939468383789 -40.2161750793457 -36.441810607910156 -46.28669357299805 -46.286712646484375
9.350934028625488 -20.598819732666016 -39.28305435180664 -50.34032440185547 -35.889583587646484 -35.889583587646484
9.46469783782959 -20.481962203979492 -39.51419448852539 -46.94646453857422 -39.840660095214844 -39.840667724609375
9.571065902709961 -21.17789077758789 -40.32975387573242 -30.68022918701172 -36.1733

...

8.60644245147705 -4.094601631164551 -7.889911651611328 -5.87567138671875 -4.8869171142578125 -4.8868560791015625
8.65014362335205 -3.99611496925354 -8.027177810668945 -7.877197265625 -11.001495361328125 -11.001953125
8.679316520690918 -4.176912307739258 -8.118416786193848 -5.798248291015625 -11.800582885742188 -11.80035400390625
8.645956039428711 -4.31675386428833 -8.52100944519043 -14.175491333007812 -6.1284637451171875 -6.1284637451171875
8.662912368774414 -4.272984027862549 -8.354443550109863 -11.196502685546875 -5.9764404296875 -5.976409912109375
8.622897148132324 -4.187812805175781 -8.301139831542969 -3.668182373046875 -8.216796875 -8.21673583984375
8.560224533081055 -4.145314693450928 -8.306785583496094 -13.593063354492188 -2.45477294921875 -2.454925537109375
8.510711669921875 -4.100499153137207 -8.07463264465332 -5.108856201171875 -11.23626708984375 -11.236572265625
8.579292297363281 -4.233442306518555 -8.343392372131348 -4.37255859375 -8.40704345703125 -8.4072265625
8.495533943

8.56109619140625 -5.299139022827148 -10.437418937683105 -10.038894653320312 -14.5543212890625 -14.554473876953125
8.582422256469727 -4.904862403869629 -9.841146469116211 -7.547882080078125 -9.073638916015625 -9.073699951171875
8.5088529586792 -4.701636791229248 -9.290243148803711 -6.15850830078125 -5.63592529296875 -5.63592529296875
8.527446746826172 -4.487671375274658 -8.91816234588623 -14.09564208984375 -9.54522705078125 -9.545379638671875
8.554819107055664 -4.365606307983398 -8.625757217407227 -6.4938507080078125 -10.838623046875 -10.839019775390625
8.613020896911621 -3.869401216506958 -7.860751628875732 -10.4814453125 -5.083984375 -5.08392333984375
8.585626602172852 -4.014604568481445 -8.185267448425293 -8.434967041015625 -3.795684814453125 -3.795806884765625
8.607606887817383 -3.704014539718628 -7.622824668884277 -6.608600616455078 -7.0282440185546875 -7.0283355712890625
8.541019439697266 -3.760331630706787 -7.339150905609131 -10.369644165039062 -4.68280029296875 -4.682861328125
8

8.65031623840332 -4.396609306335449 -8.663978576660156 -8.241119384765625 -14.178009033203125 -14.177886962890625
8.636431694030762 -4.5271430015563965 -9.01179313659668 -5.56964111328125 -17.353179931640625 -17.353363037109375
8.617762565612793 -4.490229606628418 -8.738134384155273 -12.905059814453125 -10.608978271484375 -10.60906982421875
8.606643676757812 -4.656660079956055 -9.257591247558594 -5.3878173828125 -7.1765594482421875 -7.1767120361328125
8.60032844543457 -4.393994331359863 -8.873766899108887 -6.679168701171875 -5.802734375 -5.802886962890625
8.525738716125488 -4.374027729034424 -8.665563583374023 -9.413909912109375 -13.240203857421875 -13.2401123046875
8.54595947265625 -4.341591835021973 -8.296818733215332 -3.2884521484375 -6.76922607421875 -6.769012451171875
8.587112426757812 -4.425106048583984 -8.87326717376709 -6.50543212890625 -7.38116455078125 -7.381134033203125
8.540068626403809 -4.2475504875183105 -8.568205833435059 -4.5918121337890625 -8.0126953125 -8.012634277343

8.476317405700684 -4.276827812194824 -8.30269718170166 -4.880859375 -7.1307373046875 -7.130767822265625
8.505377769470215 -4.147331714630127 -8.146973609924316 -7.984100341796875 -7.581512451171875 -7.5811767578125
8.50943374633789 -4.277655601501465 -8.159610748291016 -5.001190185546875 -3.20166015625 -3.201690673828125
8.539363861083984 -4.313971519470215 -8.577042579650879 -6.630462646484375 -13.831985473632812 -13.831741333007812
8.550606727600098 -4.325618267059326 -8.582067489624023 -8.986343383789062 -8.478515625 -8.47845458984375
8.472455978393555 -4.389607906341553 -8.713776588439941 -8.857269287109375 -5.740631103515625 -5.74053955078125
8.49632740020752 -4.536314964294434 -8.787994384765625 -5.410247802734375 -8.36041259765625 -8.360504150390625
8.456690788269043 -4.378264427185059 -8.677370071411133 -9.7762451171875 -3.18487548828125 -3.18487548828125
8.491634368896484 -4.619201183319092 -8.773510932922363 -5.96185302734375 -8.532180786132812 -8.5322265625
8.498546600341797

KeyboardInterrupt: 

In [ ]:
def evaluate(epochs = 30000, npoints = 10, batchsize = 100, nsamples = 8, negative_cutoff = 1):
    NN.eval()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        
        NN.eval()
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
     

In [ ]:
evaluate(10, 50, 1, 2)

In [6]:
%%bash
nvidia-smi
free -m

Wed Mar 17 23:58:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.118.02   Driver Version: 440.118.02   CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:86:00.0 Off |                    0 |
| N/A   57C    P0   157W / 250W |  16229MiB / 16280MiB |     88%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   31C    P0    32W / 250W |   4747MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        